In [6]:
from Backtester.streamer.simple import SimpleStreamer

class SimpleIndicator(SimpleStreamer):
    def __init__(self, exchange="bybit", symbol="BTC/USDT:USDT"):
        super().__init__(exchange, symbol)

    def get_vwap(self):
        pass

In [3]:
import numpy as np
import pandas as pd
from Backtester.streamer.simple import SimpleStreamer
simp = SimpleIndicator()

In [4]:
def calc_vwap(candles: np.ndarray) -> float:
    vol_avg = ((candles[:,2] + candles[:,3]) / 2) * candles[:,5]
    print(np.sum(vol_avg), np.sum(candles[:,5]))
    return np.sum(vol_avg) / np.sum(candles[:,5])

In [10]:
[start, end, *_] = simp._date_iterator("2022-12-01",  "2022-12-02", "1D", astype="int")

In [11]:
meta, klines = simp.get(start=start, end=end)

{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1669852800000, 'end': 1669939200000, 'limit': 1440}


In [14]:
from Backtester.database.models.candle import Candle
from peewee import fn


#meta = {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1669852800000, 'end': 1669939200000, 'limit': 1440}

def query_vwap(meta ,db=Candle):
    _db = db.alias()

    vw_p = (_db.high + _db.low)/2 * _db.volume
    vwp_sum = _db.select(fn.SUM(vw_p).alias("vwp_sum")).where(_db.exchange == meta["exchange"],
                                                   _db.symbol == meta["market"],
                                                   _db.timeframe == meta["timeframe"],
                                                   _db.timestamp.between(meta["start"], meta["end"])).scalar()

    v_sum = db.select(fn.SUM(db.volume)).where(db.exchange == meta["exchange"],
                                               db.symbol == meta["market"],
                                               db.timeframe == meta["timeframe"],
                                               db.timestamp.between(meta["start"], meta["end"])).scalar()
    return vwp_sum / v_sum



query_vwap(meta)

1971313137.79045 / 115583.125


17055.371515439216

In [4]:
import pandas as pd
import numpy as np

def candles_to_df(candles, columns=["Date", "Open","High","Low","Close","Volume"]): #["date", "open","high","low","close","volume"]
    df = pd.DataFrame(candles, columns=columns)
    df[columns[0]] = pd.to_datetime(df[columns[0]], unit="ms")
    return df

def get_vwap_by_tx(candles: dict)-> dict:
    vwaps = {}
    for tx, c in candles.items():
        vwaps[tx] = calc_vwap(c)
    return vwaps

def show_candles(tx: str, candles: np.ndarray):
    df = candles_to_df(candles)
    print(f"CANDLES FOR TIMEFRAME:   {tx}  -------->")
    print("\n")
    print(df.head(10))
    print("\n\n\n")

def calc_mean_price(candles: np.ndarray, **kwargs)-> np.ndarray:
    return (candles[:, 2] + candles[:, 3])/2

def vol_mean_price(candles: np.ndarray, **kwargs)-> np.ndarray:
    return ((candles[:,2] + candles[:,3]) / 2) * candles[:,5]

def calc_vwap_cum(candles: np.ndarray, **kwargs)-> np.ndarray:
    vol_cum = np.cumsum(candles[:,5])
    vol_avg_cum = np.cumsum(vol_mean_price(candles))
    return vol_avg_cum / vol_cum

def vwap_delta(mean, vwap_cum):
    return mean - vwap_cum

def vwap_deviation(vwap_delta):
    return abs(vwap_delta)**2

def vwap_std(vwap_dev):
    return np.sqrt(vwap_dev)

def get_vwap_std(candles: np.ndarray):
    _mean = calc_mean_price(candles)
    _vwap_cum = calc_vwap_cum(candles)
    return vwap_std(vwap_deviation(vwap_delta(_mean, _vwap_cum)))

def calc_vwap_state(candles, **kwargs):
    mean = calc_mean_price(candles)
    vwap = calc_vwap_cum(candles)
    std = calc_std_cum(candles)
    return np.divide((mean - vwap), std, out=np.zeros(std.shape, dtype=float), where=std!=0)
    #return np.where(std != 0, (mean - vwap) / std, 0)

def get_bands(_mean, _std, factor=1, **kwargs):
    hh_band = _mean + factor*_std
    ll_band = _mean - factor*_std
    return hh_band, ll_band

def calc_std_cum(candles: np.ndarray, **kwargs):
    vw_std = get_vwap_std(candles)
    return np.cumsum(vw_std) / np.arange(1,len(vw_std)+1)


def get_vwap_eval_df(c: np.ndarray):
    _std = calc_std_cum(c)
    _vwap = calc_vwap_cum(c)
    _mean = calc_mean_price(c)
    _state = calc_vwap_state(c)

    hb1, lb1 = get_bands(_vwap, _std)
    hb2, lb2 = get_bands(_vwap, _std, 2)

    df = pd.DataFrame({"std":_std, "vwap":_vwap, "price":_mean, "high_band":hb1, "low_band":lb1, "high_band2":hb2, "low_band2":lb2, "std_state":_state}, index=c[:,0])
    df.index = pd.to_datetime(df.index, unit="ms")
    df.index.name = "Date"
    return df

In [7]:
from Backtester.streamer.simple import SimpleStreamer
simp = SimpleIndicator()

meta, candles = simp.get(start="2022-12-01",end="2022-12-02")

{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1669852800000, 'end': 1669939200000, 'limit': 1440}


In [8]:
get_vwap_eval_df(np.array(candles))

,std,vwap,price,high_band,low_band,high_band2,low_band2,std_state
Date,,,,,,,,
2022-12-01 00:00:00,0.000000,17148.000000,17148.00,17148.000000,17148.000000,17148.000000,17148.000000,0.000000
2022-12-01 00:01:00,3.220053,17150.309894,17156.75,17153.529947,17147.089841,17156.750000,17143.869788,2.000000
2022-12-01 00:02:00,2.234991,17150.264866,17150.00,17152.499856,17148.029875,17154.734847,17145.794885,-0.118509
2022-12-01 00:03:00,2.643931,17149.370754,17145.50,17152.014685,17146.726822,17154.658616,17144.082891,-1.464014
2022-12-01 00:04:00,2.377651,17149.687471,17151.00,17152.065122,17147.309821,17154.442773,17144.932170,0.552027
...,...,...,...,...,...,...,...,...
2022-12-01 23:56:00,64.752997,17055.537866,16955.00,17120.290863,16990.784869,17185.043860,16926.031871,-1.552636
2022-12-01 23:57:00,64.775615,17055.527749,16958.25,17120.303365,16990.752134,17185.078980,16925.976519,-1.501765
2022-12-01 23:58:00,64.798026,17055.525228,16958.50,17120.323255,16990.727202,17185.121281,16925.929175,-1.497349


In [21]:
pd.DataFrame(list(res.dicts()))

,tms,std,hband,lband,hband2,lband2
0,1669852800000,0.000000,17148.000000,17148.000000,17148.000000,17148.000000
1,1669852860000,3.220322,17153.529678,17147.089034,17156.750000,17143.868712
2,1669852920000,2.235093,17152.499729,17148.029543,17154.734822,17145.794449
3,1669852980000,2.644055,17152.014995,17146.726885,17154.659050,17144.082830
4,1669853040000,2.377659,17152.065582,17147.310263,17154.443242,17144.932604
...,...,...,...,...,...,...
1436,1669938960000,64.750745,17120.286441,16990.784952,17185.037186,16926.034208
1437,1669939020000,64.773362,17120.298647,16990.751922,17185.072010,16925.978560
1438,1669939080000,64.795773,17120.318241,16990.726695,17185.114015,16925.930922
1439,1669939140000,64.812886,17120.251696,16990.625923,17185.064583,16925.813037


In [10]:
_vwap_cum = get_vwap_std(np.array(candles))
_vwap_cum

array([ 0.        ,  6.4401059 ,  0.26486565, ..., 97.02522817,
       89.4413272 , 85.37363462])

In [8]:
a = np.array(res.tuples())[:, 1]
a / np.arange(1, len(a) +1)

array([ 0.        ,  3.22032194,  2.2350932 , ..., 64.79577325,
       64.81288647, 64.82715339])

In [12]:
import pandas as pd
import numpy as np
df = pd.DataFrame(list(res.dicts()))

In [13]:
df

,tms,idx,std
0,1669852800000,1,0.000000
1,1669852860000,2,6.440644
2,1669852920000,3,6.705280
3,1669852980000,4,10.576220
4,1669853040000,5,11.888297
...,...,...,...
1436,1669938960000,1437,93046.819950
1437,1669939020000,1438,93144.095235
1438,1669939080000,1439,93241.117703
1439,1669939140000,1440,93330.556512


In [1]:
meta = {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1669852800000, 'end': 1669939200000, 'limit': 1440}

In [11]:
from Backtester.database.models.candle import Candle
def query_vwap(meta ,db=Candle):
    from peewee import Window, fn
    _db = db.alias()

    p_avg = (db.high + db.low)/2
    vwp = p_avg * db.volume
    vwap_cs = fn.SUM(vwp).over(order_by=[db.timestamp]) / fn.SUM(db.volume).over(order_by=[db.timestamp])
    vwap_d = p_avg - vwap_cs
    vwap_std = fn.SQRT(vwap_d*vwap_d)



    vwap_cstd = fn.SUM(vwap_std).over(order_by=[db.timestamp])

    hband_one = vwap_cs + vwap_cstd * 1
    lband_one = vwap_cs - vwap_cstd *1
    hband_two = vwap_cs + vwap_cstd * 2
    lband_two = vwap_cs - vwap_cstd * 2

    vwap_cumsum = db.select(db.timestamp, vwap_cstd.alias("std"), vwap_cs.alias("vwap"), p_avg.alias("price"), hband_one.alias("hband"), lband_one.alias("lband"), hband_two.alias("hband2"), lband_two.alias("lband2")).where(db.exchange == meta["exchange"],
                                               db.symbol == meta["market"],
                                               db.timeframe == meta["timeframe"],
                                               db.timestamp.between(meta["start"], meta["end"]))
    return vwap_cumsum

res =query_vwap(meta)

In [28]:
pd.to_datetime(1669852800000, unit="ms")

Timestamp('2022-12-01 00:00:00')

In [31]:
np.array(list(res.tuples()))

array([[1.66985280e+12, 1.71480000e+04, 3.07004070e+06, 3.07004070e+06,
        1.79032000e+02, 3.06986166e+06],
       [1.66985286e+12, 1.71567500e+04, 1.10170349e+06, 4.17174419e+06,
        2.43246000e+02, 4.17150094e+06],
       [1.66985292e+12, 1.71500000e+04, 7.09203961e+05, 4.88094815e+06,
        2.84599000e+02, 4.88066355e+06],
       ...,
       [1.66993908e+12, 1.69585000e+04, 5.08415814e+04, 1.96792281e+09,
        1.15383320e+05, 1.96780743e+09],
       [1.66993914e+12, 1.69660000e+04, 1.83633198e+06, 1.96975914e+09,
        1.15491555e+05, 1.96964365e+09],
       [1.66993920e+12, 1.69700000e+04, 1.55399378e+06, 1.97131314e+09,
        1.15583125e+05, 1.97119755e+09]])

In [20]:
pd.DataFrame(list(res.dicts()))

,tms,std,hband,lband,hband2,lband2
0,1669852800000,0.000000,17148.000000,17148.000000,17148.000000,17148.000000
1,1669852860000,3.220322,17153.529678,17147.089034,17156.750000,17143.868712
2,1669852920000,2.235093,17152.499729,17148.029543,17154.734822,17145.794449
3,1669852980000,2.644055,17152.014995,17146.726885,17154.659050,17144.082830
4,1669853040000,2.377659,17152.065582,17147.310263,17154.443242,17144.932604
...,...,...,...,...,...,...
1436,1669938960000,64.750745,17120.286441,16990.784952,17185.037186,16926.034208
1437,1669939020000,64.773362,17120.298647,16990.751922,17185.072010,16925.978560
1438,1669939080000,64.795773,17120.318241,16990.726695,17185.114015,16925.930922
1439,1669939140000,64.812886,17120.251696,16990.625923,17185.064583,16925.813037


In [19]:
from Backtester.database.models.candle import Candle
def query_vwap(meta ,_db=Candle):
    from peewee import Window, fn
    db = _db.alias()

    p_avg = (db.high + db.low)/2
    row = 1
    rows = fn.SUM(row).over(order_by=[db.timestamp])
    vwp = p_avg * db.volume
    vwap_cs = fn.SUM(vwp).over(order_by=[db.timestamp]) / fn.SUM(db.volume).over(order_by=[db.timestamp])
    vwap_d = p_avg - vwap_cs
    vwap_std = fn.SQRT(vwap_d*vwap_d)

    subq = db.select(db.timestamp.alias("tms"),
                     rows.alias("idx"),
                     vwap_std.alias("_std"),
                     vwap_cs.alias("vwap"),
                     p_avg.alias("price")).where(db.exchange == meta["exchange"],
                                                      db.symbol == meta["market"],
                                                      db.timeframe == meta["timeframe"],
                                                      db.timestamp.between(meta["start"], meta["end"]))

    std = fn.SUM(subq.c._std).over(order_by=[subq.c.tms]) / subq.c.idx
    hband_one = subq.c.vwap + std * 1
    lband_one = subq.c.vwap - std *1
    hband_two = subq.c.vwap + std * 2
    lband_two = subq.c.vwap - std * 2

    resq = _db.select(subq.c.tms,
                      std.alias("std"),
                      hband_one.alias("hband"),
                      lband_one.alias("lband"),
                      hband_two.alias("hband2"),
                      lband_two.alias("lband2")).from_(subq)
                     # subq.c.idx,
                     #  fn.SUM(subq.c._std).over(order_by=[subq.c.tms]).alias("std")).from_(subq)

    return resq

res = query_vwap(meta)
                     #
                     #
                     #
                     #
                     # vwap_cs.alias("vwap"),
                     # p_avg.alias("price"),
                     # hband_one.alias("hband"),
                     # lband_one.alias("lband"),
                     # hband_two.alias("hband2"),
                     # lband_two.alias("lband2")).where(db.exchange == meta["exchange"],
                     #                                  db.symbol == meta["market"],
                     #                                  db.timeframe == meta["timeframe"],
                     #                                  db.timestamp.between(meta["start"], meta["end"]))
    #
    # vwap_cstd = fn.SUM(vwap_std).over(order_by=[db.timestamp])
    #

    #
    # vwap_cumsum = db.select(db.timestamp, vwap_cstd.alias("std"), vwap_cs.alias("vwap"), p_avg.alias("price"), hband_one.alias("hband"), lband_one.alias("lband"), hband_two.alias("hband2"), lband_two.alias("lband2")).where(db.exchange == meta["exchange"],
    #                                                                                                                                                                                                                            db.symbol == meta["market"],
    #                                                                                                                                                                                                                            db.timeframe == meta["timeframe"],
    #                                                                                                                                                                                                                            db.timestamp.between(meta["start"], meta["end"]))
    # return vwap_cumsum

In [1]:
meta = {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '1m', 'start': 1669852800000, 'end': 1669939200000, 'limit': 1440}

In [9]:
def query_drawdown(meta: dict, db=None, direction="buy"):
    from Backtester.database.models.candle import Candle
    from peewee import Window, fn
    db = Candle if not db else db


    if direction == "buy":
        minq = db.select(fn.MIN(db.low)).where(db.exchange == meta["exchange"],
                                                db.symbol == meta["market"],
                                                db.timeframe == meta["timeframe"],
                                                db.timestamp.between(meta["start"], meta["end"]))

        query = db.select(db.timestamp, db.low).where(db.exchange == meta["exchange"],
                                                      db.symbol == meta["market"],
                                                      db.timeframe == meta["timeframe"],
                                                      db.timestamp.between(meta["start"], meta["end"]),
                                                      db.low == minq)

    else:
        maxq = db.select(fn.MAX(db.high)).where(db.exchange == meta["exchange"],
                                                                db.symbol == meta["market"],
                                                                db.timeframe == meta["timeframe"],
                                                                db.timestamp.between(meta["start"], meta["end"]))
        query = db.select(db.timestamp, db.high).where(db.exchange == meta["exchange"],
                                                      db.symbol == meta["market"],
                                                      db.timeframe == meta["timeframe"],
                                                      db.timestamp.between(meta["start"], meta["end"]),
                                                      db.high == maxq)
    return query

In [10]:
res = query_drawdown(meta)

In [11]:
list(res.tuples())

[(1669928520000, 16850.0)]